<h3>Tratar valores nulos nos dados de gatos utilizando média de outro dataset</h3>

In [1]:
import pandas as pd
dataset_dogs = pd.read_csv('../data/NYC_Dog_Licensing_Dataset.csv')

<b>Removendo colunas que não iremos utilizar</b>

In [2]:
dataset_dogs.drop(['RowNumber', 'CensusTract2010', 'NTA', 'CityCouncilDistrict', 'CommunityDistrict', 'CongressionalDistrict', 'StateSenatorialDistrict', 'LicenseIssuedDate', 'LicenseExpiredDate'], axis=1, inplace=True)
dataset_dogs.drop(dataset_dogs.index[67146:122203], axis=0, inplace=True)

In [3]:
from datetime import datetime

# substituindo os valores do campo AnimalBirthMonth para retirar a hora deixa somente a data
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'], format="%d/%m/%Y %H:%M:%S %p")
dataset_dogs['AnimalBirthMonth'] = pd.to_datetime(dataset_dogs['AnimalBirthMonth'])
dataset_dogs['AnimalBirthMonth'] = dataset_dogs['AnimalBirthMonth'].map(lambda x: datetime.strftime(x,"%d/%m/%Y"), na_action='ignore')

In [4]:
dataset_dogs.to_csv('../data/dogs_clean.csv')

In [5]:
import os

dir_atual = os.getcwd()
dir_destino = os.path.abspath(os.path.join(dir_atual, os.pardir, '/Users/kalibeg/Documents/Faculdade/grupo11-4cco-tcc/src/inteligencia_artificial/train'))

os.chdir(dir_destino)

arquivos_jpg = [f for f in os.listdir() if f.endswith('.jpg')]
fotos = set(arquivos_jpg)

In [6]:
dataset_breed_dogs = pd.read_csv('../../data/labels.csv')

In [7]:
import base64

def is_base64(s):
    try:
        base64.b64decode(s)
        return True
    except Exception:
        return False

In [8]:
dataset_dogs['Image'] = None

In [9]:
dataset_dogs.head(3)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Image
0,SHADOW,M,01/01/2000,Beagle,Brooklyn,11236.0,None
1,ROCCO,M,10/01/2011,Boxer,Brooklyn,11210.0,None
2,LUIGI,M,09/01/2005,Maltese,Bronx,10464.0,None


In [10]:
import base64
from PIL import Image
from tqdm import tqdm

def find_image_from_name_dog(breed_name):
    if breed_name == 'Unknown' or not breed_name:
        return None, dataset_breed_dogs
    breed_name_lower = str(breed_name).lower()
    breeds_founded = dataset_breed_dogs[dataset_breed_dogs['breed'].str.lower().str.contains(breed_name_lower, case=False, regex=False)]
    
    if breeds_founded.empty:
        return None, dataset_breed_dogs
    
    first_breed = breeds_founded.iloc[0]
    df = dataset_breed_dogs.drop(breeds_founded.index[0])
    info_breed = first_breed.to_dict()
    indice_arquivo = arquivos_jpg.index(f"{info_breed['id']}.jpg")
    nome_arquivo = arquivos_jpg[indice_arquivo]
    caminho_completo = os.path.join(dir_destino, nome_arquivo)
    img = Image.open(caminho_completo)
    img_bytes = img.tobytes()
    return base64.b64encode(img_bytes).decode('utf-8'), df

try:
    for index, row in tqdm(dataset_dogs.iterrows(), total=len(dataset_dogs)):
        image_base64, df = find_image_from_name_dog(row['BreedName'])
        dataset_breed_dogs = df
        if not is_base64(row["Image"]):
            dataset_dogs.loc[index, 'Image'] = image_base64
except Exception as error:
    print(error)

dataset_dogs.to_csv('../../data/dogs_clean.csv')

100%|██████████| 67146/67146 [02:28<00:00, 451.65it/s]


In [11]:
dataset_breed_dogs.tail(2)

,id,breed
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher
10221,fff43b07992508bc822f33d8ffd902ae,chesapeake_bay_retriever


In [12]:
dataset_dogs.head(2)

,AnimalName,AnimalGender,AnimalBirthMonth,BreedName,Borough,ZipCode,Image
0,SHADOW,M,01/01/2000,Beagle,Brooklyn,11236.0,SWcnFzUAI0AIHTkIABIAHTkTCyUIGzUaABwFABsCEzEXBS...
1,ROCCO,M,10/01/2011,Boxer,Brooklyn,11210.0,KColKColKColJigjISMeGx0YFRcSERMODxEMDxEMDhALDh...
